In [1]:
!pip install transformers
!pip install torch


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 62.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 59.2 MB/s eta 0:00:00


In [2]:
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer

In [3]:
import csv
import pandas as pd

In [4]:
# Load the pre-trained BERT model.
model = AutoModelForSequenceClassification.from_pretrained("mrm8488/bert-tiny-finetuned-sms-spam-detection")

In [5]:
# Load the BERT tokenizer.
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [6]:
# Define a function to classify an email.
def classify_email(email):
    """Classifies an email as spam or not spam.

    Args:
        email: A string containing the email to classify.

    Returns:
        A string indicating whether the email is spam or not spam.
    """

    tokens = tokenizer(email, return_tensors="pt")
    input_ids = tokens.get('input_ids')

    logits = model(input_ids)
    logits_tensor = logits.logits
    predictions = torch.argmax(logits_tensor, dim=-1)

    if predictions == 0:
        return "spam"
    else:
        return "ham"

In [7]:
# Read the CSV file containing the emails.
emails_df = pd.read_csv("SpamHamPreprocessedDataset.csv")

In [7]:
emails_df

In [8]:
import torch
# Add a new column to the DataFrame to store the spam/ham classification.
emails_df["spam_or_ham"] = emails_df["Generatedbody"].apply(classify_email)

In [9]:
# Save the updated DataFrame to a new CSV file.
emails_df.to_csv("EmailClassification_mrm8488 _ bert-tiny-finetuned-sms-spam-detection.csv", index=False)